In [ ]:
from pydantic import BaseModel 
from typing import List 
import json 
from config import settings
from chromadb import Client
import logging 
from chromadb import PersistentClient
from sentence_transformers import SentenceTransformer
from chromadb.utils import embedding_functions
from helper import normalize_query as n 
from chromadb_handeler import ChromaDBHandler
import hashlib 
from ask_ai import ask 


try : 
    from config import APPConfig, settings

except : 
    print("coud't not import settigns config.py so using dummy settings")

    class APPConfig:
        CHROMA_PERSIST_DIR = "./chroma_db_dummy"
        COLLECTION_NAME = "semantic_cache_dummy"
        EMBEDDING_MODEL_NAME = "BAAI/bge-base-en-v1.5"


    settings= APPConfig()


class AiOutput(BaseModel):
    """ FAQ Bot  """

class Semantic_Cache:
    def __init__(self, config: APPConfig):

        self.config= config 
        ## initilized ChromaDBHandler 
        try : 
            self.db_handler= ChromaDBHandler(
                persist_dir=self.config.CHROMA_PERSIST_DIR , 
                collection_name=self.config.COLLECTION_NAME , 
                model_name=self.config.EMBEDDING_MODEL_NAME
            )

            self.cliet= self.db_handler.client 
            self.index= self.db_handler.collection 

        except Exception as e :
            raise "chromaDBhander not initilized"
        
        ## initilized sentence transformer
        self.model= SentenceTransformer(self.config.EMBEDDING_MODEL_NAME)

    def _get_id(self, query):
        normalized_query= n(query) 
        return hashlib.sha256(normalized_query.encode('utf-8')).hexdigest()
    
    def add_entry(self, entry_id:str , response_text:str , embeddings: List[float]):

        try : 
            """ we add cache i.e query embeddings , and response with query id(hash) in this 
            using UPSERT which is (update if already exist) else insert  """

            self.index.upsert(
                ids=[self.entry_id],
                embeddings=[embeddings],
                metadatas=[{'response':response_text}])
            
            print("update sucessfull")

        except Exception  as e : 
            raise "error on add_entry"
        
    def reset_cache(self):
        self.db_handler.reset_collection()
        self.index= self.db_handler.collection

    def _get_by_id(self,entry_id):
        try :
            ## direct search 
            direct_search= self.index.get(ids=[entry_id], include=['metadatas'])
            if direct_search['ids']:
                print("Cache fould though direct  lookup")
                return direct_search 
        
        except Exception as e : 
            print("error on get_by_id")
            return None 
                
    
    def _search_by_embedding(self, query:str, query_embedding:List[float],  threshold:float = 0.92):
        self.query_embedding= self.model.encode(query).tolist()

        ## search the index 
        results= self.index.query(query_embeddings=[query_embedding], n_results=1 , 
                                 include=['metadatas', 'distances'])
        
        ## check distance against the threeshold 
        if (results and 
            results['distances']and 
            results['distances'][0] and 
            results['distances'][0][0] <= (1- threshold)): ## lower the distance more the similiraty

                    response= results['metadatas'][0][0]['response']
                    distance= results['distances'][0][0]

                    print(f"Semantic Cachhe Hit")
                    return response 
        
        return None 
    

    def cache_check(self, query:str):

        """ Perform two step cache look up
          1) exact ID match 
          2) semantic similarity search 
          if answer is not found on both then , ask to ai and store embeddings and answers for next time"""
        
        self.entry_id= self._get_id(query= query)
        exact_match= self._get_by_id(self.entry_id) ## exact match 
        if exact_match:
            return exact_match 
        
        embedded_query=self.model.encode(query).tolist()

        semantic_response= self._search_by_embedding(query , query_embedding= embedded_query ,  threshold= 0.92)
        if semantic_response: 
            return semantic_response
        
        ## Cache miss , ask llm 
        aiasnwer= ask(query=query)
        aiasnwer= aiasnwer.response
        

        ### save to entry point
        self.add_entry(entry_id= self.entry_id , response_text=aiasnwer , embeddings=embedded_query )
        return aiasnwer




    


/home/ujjwal/Desktop/FAQ-Bill-Saver-ai/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ass = Semantic_Cache(config=settings)

In [3]:
reply1= ass.cache_check("what is the capital of nepal")

Cache fould though direct  lookup


In [4]:
reply1

{'ids': ['e13b0c1fb05ca07686d35c64aa410ae75c066516fecadcec99aec1eb4f648a5c'],
 'embeddings': None,
 'documents': None,
 'uris': None,
 'included': ['metadatas'],
 'data': None,
 'metadatas': [{'response': 'donot response anyother thing asked other than , Aws and Aws SageMaker'}]}

In [8]:
ass.cache_check("what capital nepal?")

error on get_by_id


NotFoundError: Error getting collection: Collection [5e4ab8c7-9d36-428f-8d22-ffe4dcfd9607] does not exist.

'5515cca458a0fa617d941df7796e87d4c6a6e29c9d3ea89ab35d657a37969891'

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

# Use the BAAI/bge-base-en-v1.5 model
# ChromaDB will download this model for local use.
model_name = "BAAI/bge-base-en-v1.5"
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=model_name
)

# NOTE: For BGE models, you can optionally include a query instruction
# for improved retrieval, though the BGE-v1.5 documentation suggests
# it's often not strictly necessary for general retrieval.
# query_instruction="Represent this sentence for searching relevant passages:"

In [8]:
client = chromadb.Client() # Creates an in-memory client for quick testing
# For a persistent client (saves data to disk):
# client = chromadb.PersistentClient(path="./my_chroma_db")

collection = client.get_or_create_collection(
    name="my_bge_collection", 
    embedding_function=sentence_transformer_ef # Pass your function here
)

In [9]:
documents = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium."
]

collection.add(
    documents=documents,
    ids=["doc1", "doc2", "doc3"],
    metadatas=[{"source": "a"}, {"source": "b"}, {"source": "c"}]
)

print(f"Collection count: {collection.count()}") 
# Output: Collection count: 3 (Each document now has a BGE embedding)

Collection count: 3


In [10]:
results = collection.query(
    query_texts=["Where is the car going?"],
    n_results=1
)

# The result should be closest to "He drove to the stadium."
print(results['documents'])
# Example Output: [['He drove to the stadium.']]

[['He drove to the stadium.']]
